In [ ]:
# download GAIN
!git clone git@github.com:jsyoon0823/GAIN.git
# upgrade GAIN to tensorflow v2, tf.placeholder is no longer working in v2 etc.
!tf_upgrade_v2 --infile GAIN/gain.py --outfile GAIN/gain_tf2.py
!tf_upgrade_v2 --infile GAIN/utils.py --outfile GAIN/utils_tf2.py

In [ ]:
# rename utils to .utils_tf2 in GAIN/gain_tf2.py

In [5]:
import pandas as pd
import numpy as np
from GAIN.gain_tf2 import gain
from GAIN.utils_tf2 import binary_sampler
from GAIN.utils_tf2 import rmse_loss

# Create missing data in TEP dataset

In [3]:
tep_dataset = pd.read_csv('Tennessee_Event-Driven/datasets/tep_extended_dataset_simrun1.csv.csv',index_col=False)

In [4]:
dataset_X = tep_dataset.drop(columns=['faultNumber', 'simulationRun', 'sample']).values
dataset_Y = tep_dataset['faultNumber'].values

In [5]:
no, dim = dataset_X.shape
p = 0.1
# Introduce missing data
mask = binary_sampler(1-p, no, dim)
dataset_X_missing = dataset_X.copy()
dataset_X_missing[mask == 0] = np.nan

# Impute data

In [21]:
gain_parameters = {'batch_size': 128,
                 'hint_rate': 1.5,
                 'alpha': 100,
                 'iterations': 1000}

In [22]:
dataset_X_imputed = gain(dataset_X_missing, gain_parameters)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 168.12it/s]


In [23]:
rmse_loss(dataset_X, dataset_X_imputed, mask)

0.07301131265367386